

**1.   Intall Dependencies**



In [ ]:
!pip install tensorflow
!pip install tensorflow-addons
!pip list
!unzip -q path_to_your_zip.zip
!pip install Pillow
!pip install opencv-contrib-python



**2.   Import Libraries**



In [2]:
import tensorflow as tf
import os
import imghdr
import cv2
import zipfile
import shutil
import numpy as np
import matplotlib.pyplot as plt
import traceback
from google.colab import files
import zipfile
from PIL import Image
import io




**3. Image Preprocessing**



In [3]:
def auto_canny(image, sigma=0.1):

    # Convert the PIL image to a NumPy array
    image = np.array(image)

    # Resize image
    image = cv2.resize(image, (224, 224))

    # Convert the image to grayscale if it's not already
    if len(image.shape) > 2:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Perform adaptive thresholding to segment the image
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Apply automatic Canny edge detection using the computed median
    v = np.median(blurred)
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    canny = cv2.Canny(blurred, lower, upper)

    # Dilation to connect adjacent edges
    kernel = np.ones((3,3), np.uint8)
    canny = cv2.dilate(canny, kernel, iterations=1)

    # Morphological closing to connect edges
    kernel = np.ones((5, 5), np.uint8)
    closed = cv2.morphologyEx(canny, cv2.MORPH_CLOSE, kernel)

    # Applying threshold as mask
    canny = cv2.bitwise_and(closed, thresh)

    # Contour detection
    contours, _ = cv2.findContours(canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Area filtering
    min_area = 100
    max_area = 10000
    final_contours = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if min_area < area < max_area:
            final_contours.append(cnt)

    # Draw remaining contours on a new image
    contour_img = cv2.drawContours(np.zeros_like(canny), final_contours, -1, (255, 255, 255), 1)

    return contour_img

# Define folders and processed folders
base_folders = ['Training', 'Testing']
processed_folders = ['Training_Processed', 'Testing_Processed']

# Process images and save the outputs
for base_folder, processed_folder in zip(base_folders, processed_folders):
    os.makedirs(processed_folder, exist_ok=True)
    images = [image for image in os.listdir(base_folder) if image.lower().endswith(('.png', '.jpg', '.jpeg'))]
    for img in images:
        im = Image.open(os.path.join(base_folder, img))
        canny = auto_canny(im)
        cv2.imwrite(os.path.join(processed_folder, img), canny)  # Save processed image





**4.   Deep Learning**








  **4.1   CNN Model**




In [4]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from PIL import Image
import cv2


# Mapping class data from Excel
class_data = pd.read_excel('/content/ImageProcessingdataset.xlsx')  # Your Excel file
class_mapping = {'S': 0, 'A': 1, 'B': 2, 'C': 3}
class_data['Level'] = class_data['Level'].map(class_mapping)
class_dict = dict(zip(class_data['Image ID'], class_data['Level']))

def get_label(file_path):
    # If the input is a tensor, decode it
    if isinstance(file_path, tf.Tensor):
        # https://stackoverflow.com/questions/40388792/how-to-decode-a-numpy-array-of-encoded-literals-strings-in-python3-attributeerr
        file_path = file_path.numpy().decode('utf-8')

    image_id = os.path.basename(file_path).split('.')[0]  # to only consider the image_ID without the extension

    # Retrieve the label
    try:
        label = class_dict[image_id]
    except KeyError:
        print(f"Filename not found in class_dict: {image_id}")
        label = -1  # Use -1 for not existing label (there is a problem here)
    return tf.cast(label, tf.int32)


def process_path(file_path):
    # We should decode the file path, necessary when using tf.py_function
    # https://stackoverflow.com/questions/69859878/tensorflow-2-6-0-error-during-model-fit-after-using-tf-py-function-valueerror
    # https://stackoverflow.com/questions/40388792/how-to-decode-a-numpy-array-of-encoded-literals-strings-in-python3-attributeerr
    file_path = file_path.numpy().decode('utf-8')

    # Read and process the image
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=1)  # Assuming images are grayscale
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0  # Normalize images

    label = tf.py_function(func=get_label, inp=[file_path], Tout=tf.int32)
    return img, label


# Set output shapes explicitly for efficiency
def set_shapes(img, label):
    img.set_shape([224, 224, 1])
    label.set_shape([])
    return img, label



def create_dataset(file_paths):
    dataset = tf.data.Dataset.from_tensor_slices(file_paths)
    # Wrap process_path to handle tensor inputs correctly
    dataset = dataset.map(lambda x: tf.py_function(func=process_path, inp=[x], Tout=[tf.float32, tf.int32]))
    dataset = dataset.map(set_shapes)  # I needed to apply set_shapes to ensure correct tensor shapes

    # Filter out invalid labels
    dataset = dataset.filter(lambda img, label: label >= 0)

    return dataset.batch(20).prefetch(tf.data.AUTOTUNE)


# Load images and their paths
train_dir = '/content/Training_Processed'
test_dir = '/content/Testing_Processed'
train_files = [os.path.join(train_dir, fname) for fname in os.listdir(train_dir) if fname.endswith(('.jpg', '.jpeg'))]
test_files = [os.path.join(test_dir, fname) for fname in os.listdir(test_dir) if fname.endswith(('.jpg', '.jpeg'))]


# Create datasets for training and testing
train_dataset = create_dataset(train_files)
test_dataset = create_dataset(test_files)

# CNN Model Definition
def create_cnn_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(4, activation='softmax')  # Use 'softmax' for multi-class classification
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create and train the model
model = create_cnn_model()
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=test_dataset
)

Epoch 1/10
18/18 [==============================] - 66s 4s/step - loss: 1.3364 - accuracy: 0.4511 - val_loss: 1.6532 - val_accuracy: 0.4013
Epoch 2/10
18/18 [==============================] - 55s 3s/step - loss: 0.8824 - accuracy: 0.6379 - val_loss: 1.5108 - val_accuracy: 0.4145
Epoch 3/10
18/18 [==============================] - 51s 3s/step - loss: 0.5618 - accuracy: 0.7874 - val_loss: 1.7707 - val_accuracy: 0.4145
Epoch 4/10
18/18 [==============================] - 54s 3s/step - loss: 0.2607 - accuracy: 0.9109 - val_loss: 3.3309 - val_accuracy: 0.4737
Epoch 5/10
18/18 [==============================] - 55s 3s/step - loss: 0.1113 - accuracy: 0.9540 - val_loss: 4.6920 - val_accuracy: 0.4474
Epoch 6/10
18/18 [==============================] - 56s 3s/step - loss: 0.1387 - accuracy: 0.9540 - val_loss: 4.1062 - val_accuracy: 0.4671
Epoch 7/10
18/18 [==============================] - 54s 3s/step - loss: 0.0688 - accuracy: 0.9770 - val_loss: 4.1023 - val_accuracy: 0.4671
Epoch 8/10
18/18 [==



  **4.1.   ResNet Model**



In [11]:
import os
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50

# Mapping class data from Excel
class_data = pd.read_excel('/content/ImageProcessingdataset.xlsx')  # Your Excel file
class_mapping = {'S': 0, 'A': 1, 'B': 2, 'C': 3}
class_data['Level'] = class_data['Level'].map(class_mapping)
class_dict = dict(zip(class_data['Image ID'], class_data['Level']))

def get_label(file_path):
    if isinstance(file_path, tf.Tensor):
        file_path = file_path.numpy().decode('utf-8')
    image_id = os.path.basename(file_path).split('.')[0]
    try:
        label = class_dict[image_id]
    except KeyError:
        print(f"Filename not found in class_dict: {image_id}")
        label = -1
    return tf.cast(label, tf.int32)

def process_path(file_path):
    file_path = file_path.numpy().decode('utf-8')
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)  # Update to 3 channels for ResNet50
    img = tf.image.resize(img, [224, 224])
    img = tf.keras.applications.resnet50.preprocess_input(img)  # ResNet specific preprocessing
    label = tf.py_function(func=get_label, inp=[file_path], Tout=tf.int32)
    return img, label

def set_shapes(img, label):
    img.set_shape([224, 224, 3])  # Update to 3 channels
    label.set_shape([])
    return img, label

def create_dataset(file_paths):
    dataset = tf.data.Dataset.from_tensor_slices(file_paths)
    dataset = dataset.map(lambda x: tf.py_function(func=process_path, inp=[x], Tout=[tf.float32, tf.int32]))
    dataset = dataset.map(set_shapes)
    dataset = dataset.filter(lambda img, label: label >= 0)
    return dataset.batch(20).prefetch(tf.data.AUTOTUNE)

# Load images and their paths
train_dir = '/content/Training_Processed'
test_dir = '/content/Testing_Processed'
train_files = [os.path.join(train_dir, fname) for fname in os.listdir(train_dir) if fname.endswith(('.jpg', '.jpeg'))]
test_files = [os.path.join(test_dir, fname) for fname in os.listdir(test_dir) if fname.endswith(('.jpg', '.jpeg'))]

# Create datasets for training and testing
train_dataset = create_dataset(train_files)
test_dataset = create_dataset(test_files)

# ResNet50 Model Definition
def create_resnet_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze the convolutional base
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(4, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create and train the ResNet model
model = create_resnet_model()
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset)

Epoch 1/10
18/18 [==============================] - 114s 6s/step - loss: 1.9955 - accuracy: 0.3534 - val_loss: 1.6965 - val_accuracy: 0.3618
Epoch 2/10
18/18 [==============================] - 100s 6s/step - loss: 1.2405 - accuracy: 0.5661 - val_loss: 1.8989 - val_accuracy: 0.3684
Epoch 3/10
18/18 [==============================] - 108s 6s/step - loss: 0.9191 - accuracy: 0.5833 - val_loss: 1.7777 - val_accuracy: 0.3750
Epoch 4/10
18/18 [==============================] - 110s 6s/step - loss: 0.8241 - accuracy: 0.6609 - val_loss: 1.9034 - val_accuracy: 0.4211
Epoch 5/10
18/18 [==============================] - 109s 6s/step - loss: 0.7685 - accuracy: 0.6580 - val_loss: 1.9384 - val_accuracy: 0.3947
Epoch 6/10
18/18 [==============================] - 111s 6s/step - loss: 0.6920 - accuracy: 0.7126 - val_loss: 1.8426 - val_accuracy: 0.4276
Epoch 7/10
18/18 [==============================] - 109s 6s/step - loss: 0.6637 - accuracy: 0.7385 - val_loss: 1.8543 - val_accuracy: 0.4539
Epoch 8/10
18